In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import random

import pandas as pd

In [2]:
import os

In [3]:
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# !wget https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.39/linux64/chromedriver-linux64.zip
# !unzip chromedriver-linux64.zip
# !mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
# !chmod +x /usr/bin/chromedriver

--2025-05-04 01:48:02--  https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.39/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.207, 74.125.132.207, 74.125.201.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9593043 (9.1M) [application/zip]
Saving to: ‘chromedriver-linux64.zip.1’

chromedriver-linux6 100%[===================>]   9.15M  --.-KB/s    in 0.1s    

2025-05-04 01:48:02 (87.1 MB/s) - ‘chromedriver-linux64.zip.1’ saved [9593043/9593043]

Archive:  chromedriver-linux64.zip
replace chromedriver-linux64/LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chromedriver-linux64/LICENSE.chromedriver  
replace chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflati

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [6]:
from datetime import datetime

In [7]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"

In [8]:
service = Service("/usr/bin/chromedriver")
browser = webdriver.Chrome(service=service, options=options)

In [9]:
def craw_stock_news(ticker, start_date, end_date, roof_path):
  df = pd.DataFrame(data = None, columns = ['ticker','time', 'tittle', 'url'])
  browser.get("https://cafef.vn/du-lieu/tin-doanh-nghiep/"+str(ticker)+"/event.chn")
  print(browser.title)
  flag = True
  while (flag):
    li_elements = browser.find_elements(By.XPATH, "//div[@id='divEvents']/ul/li")
    for idx, li in enumerate(li_elements, start= 1):
      span = li.find_element(By.TAG_NAME, "span")
      curent_date_of_new = span.text
      curent_date_of_new_convert = datetime.strptime(curent_date_of_new, "%d/%m/%Y %H:%M")
      if (curent_date_of_new_convert < start_date):
        print("Đã hết thời gian cần tìm")
        flag = False
        break
      if curent_date_of_new_convert >= start_date and curent_date_of_new_convert <= end_date:
        time = curent_date_of_new
        url = li.find_element(By.TAG_NAME, "a").get_attribute('href')
        title = li.find_element(By.TAG_NAME, "a").text
        print('Đang lấy bài báo với thông tin như sau: ')
        print(time)
        print(title)
        print(url)
        new_row = {'ticker': ticker, 'time':time, 'tittle': title, 'url':url}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
      if (idx == len(li_elements)):
        print("Đây là phần tử cuối cùng")
        # Click vào nút xem tiếp
        try:
          btn_next_element = browser.find_element(By.XPATH, "//span[@id='spanNext']/a")
          browser.execute_script("arguments[0].click();", btn_next_element)
          sleep(random.uniform(1, 2))
          print("Cuyển qua trang tiếp theo")
        except Exception as e:
          print(f"Không thể click nút tiếp: {e}")
          flag = False
          break
  # Tạo thư mục nếu chưa tồn tại
  os.makedirs(roof_path, exist_ok=True)
  file_path = roof_path + "/" + str(ticker) + ".xlsx"
  with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)
  print("Đã lưu file excel cho cổ phiếu " + str(ticker))
  # 5. Đóng browser
  # browser.close()

In [26]:
end_date = datetime.strptime('04/06/2025 00:00', "%d/%m/%Y %H:%M")
start_date = datetime.strptime('01/06/2025 00:00', "%d/%m/%Y %H:%M")

In [27]:
craw_stock_news("ACB", start_date, end_date, roof_path="/home/Trong/Agentic_AI/model_ml/sentiment")

ACB : Tin tức doanh nghiệp niêm yết | CafeF.vn
Đang lấy bài báo với thông tin như sau: 
02/06/2025 00:00
ACB: Nghị quyết HĐQT về phương án phát hành trái phiếu riêng lẻ lần 2.2025
https://cafef.vn/du-lieu/ACB-2212106/acb-nghi-quyet-hdqt-ve-phuong-an-phat-hanh-trai-phieu-rieng-le-lan-22025.chn
Đang lấy bài báo với thông tin như sau: 
02/06/2025 00:00
ACB: Đã phát hành 669.998.687 cổ phiếu để trả cổ tức
https://cafef.vn/du-lieu/ACB-2212109/acb-da-phat-hanh-669998687-co-phieu-de-tra-co-tuc.chn
Đã hết thời gian cần tìm
Đã lưu file excel cho cổ phiếu ACB
